In [1]:
!pip install geojson
!pip install rasterio

# !pip install rioxarray


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 43.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from osgeo import gdal
import numpy as np
import statistics
import rasterio
from rasterio import features
import matplotlib.pyplot as plt
from rasterio.plot import plotting_extent
import geopandas as gpd
import fiona
from rasterio.crs import CRS
from rasterio.mask import mask
import pandas as pd
import json
import geojson

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

In [4]:
dataset1 = gdal.Open(r'/content/drive/MyDrive/WQA_data/DLdata/T_09-07-2023.tif')
dataset2 = gdal.Open(r'/content/drive/MyDrive/WQA_data/DLdata/T_20-04-2023.tif')

dataframe = pd.read_csv('/content/drive/MyDrive/WQA_data/AvantiCombined_NSR.csv')


In [5]:
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']
wavelengths = [442.7,	492.4,559.8,664.6,704.1,740.5,782.8,832.8,864.7,945.1,1373.5,1613.7,2202.4]
parameters = ["pH", "Salinity", "Ammonia"]

In [ ]:
with open(f'/content/drive/MyDrive/WQA_data/CroppedPond.geojson') as f:
    gj = geojson.load(f)

files = [r"/content/drive/MyDrive/WQA_data/DLdata/T_09-07-2023.tif"\
         ,r"/content/drive/MyDrive/WQA_data/DLdata/T_20-04-2023.tif"\
        ]
imageData = {}
dates = ['2023-07-09','2023-04-20']
Tresults = []
for index,f in zip( dates,files):
  results = {}
  for i in gj['features']:
      name = (i['properties']['Pond'])
      with rasterio.open(f) as src:
        out_image, out_transform = rasterio.mask.mask(src, [i['geometry']], crop=True)
        reshaped_image = np.transpose(out_image, (1, 2, 0))
        # if name in ['T_B3', 'T_B4', 'T_B5', 'T_B2','T_C8',"T_A1"]:
          # print(name)
          # plt.figure()
        array = reshaped_image[:,:,7]
        # print(array.shape)
        # plt.hist(array, bins='auto')
        # plt.show()
        imageData.update({f'{index}_{name}':array})
        # plt.figure()
        # plt.imshow(array)
        # plt.show()
        nir = reshaped_image[:,:,7]

In [ ]:
  # imagearray  = list(imageData.values())
  imageData

In [22]:
for key  in list( imageData.keys()):
  print(key.split('_'))


['2023-07-09', 'T', 'B7']
['2023-07-09', 'T', 'B5']
['2023-07-09', 'T', 'B6']
['2023-07-09', 'T', 'B4']
['2023-07-09', 'T', 'B3']
['2023-07-09', 'T', 'B2']
['2023-07-09', 'T', 'B1']
['2023-07-09', 'T', 'C8']
['2023-07-09', 'T', 'A8']
['2023-07-09', 'T', 'A7']
['2023-07-09', 'T', 'A6']
['2023-07-09', 'T', 'A5']
['2023-07-09', 'T', 'A4']
['2023-07-09', 'T', 'A3']
['2023-07-09', 'T', 'A2']
['2023-07-09', 'T', 'A1']
['2023-07-09', 'T', 'B8']
['2023-04-20', 'T', 'B7']
['2023-04-20', 'T', 'B5']
['2023-04-20', 'T', 'B6']
['2023-04-20', 'T', 'B4']
['2023-04-20', 'T', 'B3']
['2023-04-20', 'T', 'B2']
['2023-04-20', 'T', 'B1']
['2023-04-20', 'T', 'C8']
['2023-04-20', 'T', 'A8']
['2023-04-20', 'T', 'A7']
['2023-04-20', 'T', 'A6']
['2023-04-20', 'T', 'A5']
['2023-04-20', 'T', 'A4']
['2023-04-20', 'T', 'A3']
['2023-04-20', 'T', 'A2']
['2023-04-20', 'T', 'A1']
['2023-04-20', 'T', 'B8']


In [23]:
tempdf = dataframe[dataframe['Date of Image']=='2023-07-09']
temparray = []
for index, row in tempdf.iterrows():
  id = row['PondID'].split('_')[1]
  temparray.append(imageData[f'2023-07-09_T_{id}'])
print(tempdf.shape)

tempdf2 = dataframe[dataframe['Date of Image']=='2023-04-20'][dataframe['PondID'] !='KOKKUPADU']
print(tempdf.shape)
# temparray = []
for index, row in tempdf2.iterrows():
  try:
    id = row['PondID'].split('_')[1]
  except:
    pass
  temparray.append(imageData[f'2023-04-20_T_{id}'])

(15, 26)
(15, 26)


<ipython-input-23-5e789ab01893>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tempdf2 = dataframe[dataframe['Date of Image']=='2023-04-20'][dataframe['PondID'] !='KOKKUPADU']


In [24]:
# df = df[df.line_race != 0]
filterdf = pd.concat([tempdf,tempdf2])
df = filterdf[filterdf.PondID != 'KOKKUPADU']

In [25]:
len(temparray)

32

In [17]:
tempdf2

,PondID,pH,Salinity,Carbonate,BiCarbonate,Alkalinity,Hardness,Calcium,Magnesium,Ammonia,...,B12,B2,B3,B4,B5,B6,B7,B8,B8A,B9
73,TUPILIPALEM_A1,7.5,32,0,170,170,6300,368,1106,0.02,...,666,1376,1321,1296,1437,1501,1639,1483,1574,241
74,TUPILIPALEM_A6,8.2,31,20,150,170,6200,358,1186,0.06,...,398,1228,1153,874,1068,926,990,668,806,216
75,TUPILIPALEM_A7,8.2,31,20,150,170,6260,348,1181,0.05,...,558,1251,1198,958,1185,1042,1124,900,1168,206
76,TUPILIPALEM_A8,8.3,30,30,150,180,6060,340,1164,0.05,...,545,1216,1136,861,1125,916,964,659,989,256
77,TUPILIPALEM_B1,7.6,28,0,160,160,5600,358,1142,0.07,...,465,1277,1178,938,920,769,812,577,648,262
78,TUPILIPALEM_B2,8.5,35,30,140,170,6700,400,1266,0.07,...,788,1220,1176,880,1091,1039,1078,730,1068,189
79,TUPILIPALEM_B3,8.2,36,20,130,150,6880,424,1271,0.04,...,703,1213,1150,876,1148,1052,1131,633,944,227
80,TUPILIPALEM_B4,8.4,32,30,140,170,5800,366,1191,0.05,...,559,1220,1141,875,1021,799,848,661,658,260
81,TUPILIPALEM_B5,8.1,31,10,140,150,5660,358,1178,0.04,...,536,1225,1142,863,1034,962,1001,608,1044,254
82,TUPILIPALEM_B6,8.3,30,30,150,170,5600,344,1172,0.05,...,441,1261,1243,887,1103,1022,1074,721,946,194


In [13]:
len(df)

32

In [14]:
X, y = dataset[:, 1:-1], dataset[:, -1]
X, y = X.astype('float'), y.astype('float')
n_features = X.shape[1]
# split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# define the keras model
model = Sequential()
model.add(Dense(20, input_dim=n_features, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(10, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1, activation='linear'))
# compile the keras model
model.compile(loss='mse', optimizer='adam')
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=150, batch_size=32, verbose=2)
# evaluate on test set
yhat = model.predict(X_test)
error = mean_absolute_error(y_test, yhat)
print('MAE: %.3f' % error)

NameError: ignored

In [ ]:
X